###  Analyzing Business Data in SQL. Cost, revenue, profit

#### Task: calculate the total revenue. Keep only the records of user ID 15

```sql
-- Calculate revenue
SELECT  sum(meal_price*order_quantity) AS revenue
  FROM meals
  JOIN orders ON meals.meal_id = orders.meal_id

-- Keep only the records of customer ID 15
WHERE user_id = 15;

```

#### Task2. Write the expression for revenue.
Keep only the records of June 2018.

```sql
SELECT DATE_TRUNC('week', order_date) :: DATE AS delivr_week,
       -- Calculate revenue
       sum(order_quantity*	meal_price) AS revenue
  FROM meals
  JOIN orders ON meals.meal_id = orders.meal_id
-- Keep only the records in June 2018
WHERE DATE_TRUNC('month', order_date) :: DATE = '2018-06-01'
GROUP BY delivr_week
ORDER BY delivr_week ASC;


```

#### Task3. Calculate cost per meal ID.
Set the LIMIT to 5

```sql

SELECT
  meals.meal_id,
  sum(meal_cost*stocked_quantity) AS cost
FROM meals
JOIN stock ON meals.meal_id = stock.meal_id
GROUP BY meals.meal_id
ORDER BY cost DESC
-- Only the top 5 meal IDs by purchase cost
LIMIT 5,

```

#### Task4. Calculate cost per month.A query to calculate cost per month, wrapped in a CTE,

```sql

-- Declare a CTE named monthly_cost
WITH monthly_cost AS (
  SELECT
    DATE_TRUNC('month', stocking_date)::DATE AS delivr_month,
    SUM(meal_cost * stocked_quantity) AS cost
  FROM meals
  JOIN stock ON meals.meal_id = stock.meal_id
  GROUP BY delivr_month)

SELECT
  -- Calculate the average monthly cost before September
  avg(cost)
FROM monthly_cost
WHERE delivr_month < '2018-09-01';

```

#### Task5. Calculate revenue per eatery in the revenue CTE.
Calculate cost per eatery in the cost CTE.
Join the two CTEs and calculate profit per eatery.

```sql
WITH revenue AS (
  -- Calculate revenue per eatery
  SELECT meals.eatery,
        sum(meals.meal_price * orders.order_quantity) AS revenue
    FROM meals
    JOIN orders ON meals.meal_id = orders.meal_id
   GROUP BY eatery),

  cost AS (
    SELECT meals.eatery,
         sum(stock.stocked_quantity*meals.meal_cost) AS cost
    FROM meals
    JOIN stock ON meals.meal_id = stock.meal_id
   GROUP BY eatery)

   -- Calculate profit per eatery
   SELECT revenue.eatery,
          (revenue.revenue -  cost.cost) as profit
     FROM revenue
     JOIN cost ON revenue.eatery = cost.eatery
    ORDER BY profit DESC;
    
```

#### Task6 Calculate revenue per month in the revenue CTE.
Calculate cost per month in the cost CTE.
Join the two CTEs and calculate profit per month.

```sql
-- Set up the revenue CTE
WITH revenue AS ( 
	SELECT
		DATE_TRUNC('month', order_date) :: DATE AS delivr_month,
		sum(meals.meal_price*orders.order_quantity) AS revenue
	FROM meals
	JOIN orders ON meals.meal_id = orders.meal_id
	GROUP BY delivr_month),
-- Set up the cost CTE
  cost AS (
 	SELECT
		DATE_TRUNC('month', stocking_date) :: DATE AS delivr_month,
		sum(stock.stocked_quantity*meals.meal_cost) AS cost
	FROM meals
    JOIN stock ON meals.meal_id = stock.meal_id
	GROUP BY delivr_month)
-- Calculate profit by joining the CTEs
SELECT
	revenue.delivr_month,
	revenue.revenue - cost.cost AS profit
FROM revenue
JOIN cost ON revenue.delivr_month = cost.delivr_month
ORDER BY revenue.delivr_month ASC;

```
